# Introduction

Our project aims to identify an optimal pricing model using regression techniques to quantify what the reasonable price range of a car would be, which Atlas Motors would use for acquiring used cars for their rental fleet. Since there is complexity in determining the prices of cars due to various factors like COE, OMV and Sales Upselling, our model will reduce the frustrations and time consumption of used car purchases. 

In this study, we would be utilizing data science processes from data collection (web-scraping: BeautifulSoup, Python), data cleaning, exploratory data analysis to the model training and testing stage. The source of data comes SgCarMart, an online car sales portal in Singapore. 

In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV, ElasticNet, ElasticNetCV

import  scipy.signal.signaltools

def _centered(arr, newsize):
    # Return the center newsize portion of the array.
    newsize = np.asarray(newsize)
    currsize = np.array(arr.shape)
    startind = (currsize - newsize) // 2
    endind = startind + newsize
    myslice = [slice(startind[k], endind[k]) for k in range(len(endind))]
    return arr[tuple(myslice)]

scipy.signal.signaltools._centered = _centered
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import accuracy_score
%matplotlib inline
sns.set_style('whitegrid')

In [2]:
df_main = pd.read_csv('sgcarmart_used_cars_prices7.csv',index_col=0)
df_main.sample(5)

,LISTING_URL,BRAND,PRICE,DEPRE_VALUE_PER_YEAR,REG_DATE,MILEAGE_KM,MANUFACTURED_YEAR,ROAD_TAX_PER_YEAR,TRANSMISSION,DEREG_VALUE_FROM_SCRAPE_DATE,SCRAPE_DATE,OMV,ARF,COE_FROM_SCRAPE_DATE,DAYS_OF_COE_LEFT,ENGINE_CAPACITY_CC,CURB_WEIGHT_KG,NO_OF_OWNERS,VEHICLE_TYPE,POST_DATE
4014,https://www.sgcarmart.com/used_cars/info.php?I...,Hyundai,48800.0,15210.0,15-Nov-2016,95000.0,2016.0,738.0,Auto,22173.0,26/01/2024,12352.0,12352.0,52668.0,1009.0,1591.0,1345.0,2.0,Mid-Sized Sedan,25-Jan-2024
480,https://www.sgcarmart.com/used_cars/info.php?I...,Mercedes-Benz,139800.0,18470.0,26-Aug-2020,70047.0,2019.0,586.0,Auto,51272.0,26/01/2024,31682.0,36355.0,36502.0,2370.0,1332.0,1385.0,2.0,Hatchback,14-Jan-2024
7585,https://www.sgcarmart.com/used_cars/info.php?I...,Toyota,55800.0,20820.0,26-Jan-2016,94000.0,2015.0,1196.0,Auto,27076.0,26/01/2024,29513.0,28319.0,57501.0,395.0,1986.0,1610.0,3.0,SUV,19-Jan-2024
13006,https://www.sgcarmart.com/used_cars/info.php?I...,Jaguar,98500.0,21970.0,01-Sep-2016,NaN,2016.0,2380.0,Auto,64760.0,27/01/2024,61608.0,82895.0,57903.0,944.0,2995.0,1856.0,3.0,SUV,20-Jan-2024
11783,https://www.sgcarmart.com/used_cars/info.php?I...,Ferrari,350000.0,45060.0,06-Jan-2012,36000.0,2011.0,5297.0,Auto,47041.0,27/01/2024,272226.0,272226.0,60586.0,2829.0,4297.0,1780.0,6.0,Sports Car,23-Jan-2024


In [3]:
df_clean = df_main.drop(['LISTING_URL', 'SCRAPE_DATE'],axis=1)


In [4]:
dropped_data=df_clean.dropna()

In [5]:
dropped_data['MANUFACTURED_YEAR']

0        2015.0
2        2018.0
4        2019.0
5        2010.0
7        2018.0
          ...  
16586    2011.0
16587    2021.0
16588    2012.0
16589    2011.0
16591    2015.0
Name: MANUFACTURED_YEAR, Length: 10311, dtype: float64

In [6]:
df_clean=dropped_data
df_main['SCRAPE_DATE'] = pd.to_datetime(df_main['SCRAPE_DATE'])
df_clean['MANUFACTURED_YEAR'] = df_clean['MANUFACTURED_YEAR'].astype(int)
df_clean['CAR_AGE'] = df_main['SCRAPE_DATE'].dt.year  - df_clean['MANUFACTURED_YEAR'] # Obtaining values for age of car


C:\Users\SAMUEL~1\AppData\Local\Temp/ipykernel_22468/4274125101.py:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_main['SCRAPE_DATE'] = pd.to_datetime(df_main['SCRAPE_DATE'])


In [7]:
df_clean['POST_DATE'] = pd.to_datetime(df_clean['POST_DATE'])
df_clean['POST_AGE'] = (df_main['SCRAPE_DATE'] - df_clean['POST_DATE'])
df_clean['POST_AGE']=df_clean['POST_AGE'].dt.days

In [8]:
# Transmission conversion -> 1 for auto, 0 for manual (just 1 column only)

df_clean['TRANSMISSION_CONVERT'] = df_clean['TRANSMISSION'].apply(lambda x: 1 if x == 'Auto' else 0)
df_clean.drop('TRANSMISSION',axis=1,inplace=True)
df_clean.rename(columns={'TRANSMISSION_CONVERT':"TRANSMISSION"}, inplace=True)  # Renaming column back


In [9]:
veh_list=[]
for veh in df_clean['VEHICLE_TYPE'].unique():
    veh_list.append(veh)

veh_list.sort()
out = map(lambda x:x.lower(), veh_list)
veh_list = list(out) 


In [10]:
df_clean['VEHICLE_TYPE']
df_encoded = pd.get_dummies(df_clean['VEHICLE_TYPE'], prefix='VEHICLE_TYPE')

# Concatenating the new columns to the original DataFrame
df_encoded = df_encoded.astype(int)
df_clean = pd.concat([df_clean, df_encoded], axis=1)


## Feature Engineering: Categorization of BRAND Column


In [11]:
# Renaming Brand Names to their actual names
df_clean2=df_clean
df_clean2.loc[df_clean2['BRAND'] == 'Aston','BRAND'] = 'Aston Martin'
df_clean2.loc[df_clean2['BRAND'] == 'Land','BRAND'] = 'Land Rover'
df_clean2.loc[df_clean2['BRAND'] == 'Alfa', 'BRAND'] = 'Alfa Romeo'
# Cleaning whitespaces from the values in "Brand" to prevent any messup later
df_clean2['BRAND'].apply(str.strip)


0        Mercedes-Benz
2              Renault
4               Toyota
5                 Audi
7                Mazda
             ...      
16586    Mercedes-Benz
16587           Subaru
16588          Porsche
16589              BMW
16591            Volvo
Name: BRAND, Length: 10311, dtype: object

In [12]:
category_brands = {
    'EXOTIC': ['Koenigsegg','Bugatti','Ferrari', 'Lamborghini','Aston Martin','McLaren','Hummer'],
    'ULTRA_LUXURY': ['Porsche','Maserati','Rolls-Royce', 'Land Rover','Bentley','Maybach'],
    'LUXURY': ['MINI','Mini','Alfa Romeo','Mercedes','Mercedes-Benz', 'BMW', 'Audi', 'Lexus','Jeep','Lotus','Volvo','Peugeot','Tesla','BYD','Acura','Cadillac','Jaguar','Infiniti','Chrysler','Lincoln','Genesis'],
    'MID_LEVEL': ['Volkswagen','Renault','Ford', 'Chevrolet'],
    'ECONOMY': ['Toyota','Honda','Hyundai', 'Kia', 'Nissan', 'Mazda','Mitsubishi','Subaru','Suzuki','Citroen','Proton','Ssangyong','Daihatsu','Fiat','Skoda','Opel','MG','SEAT','Perodua'],
    'OTHERS': []  # An empty list for unspecified brands
}

# Reversing the categorization based on the 'Brand' column
df_clean2['CAR_CATEGORY'] = df_clean2['BRAND'].apply(lambda x: next((category for category, brands in category_brands.items() if x in brands), 'OTHERS'))


In [13]:
df_encoded = pd.get_dummies(df_clean2['CAR_CATEGORY'], prefix='CAR_CATEGORY')

# Concatenating the new columns to the original DataFrame
df_encoded = df_encoded.astype(int)
df_clean2 = pd.concat([df_clean2, df_encoded], axis=1)


In [14]:
df_clean_log=df_clean2.copy()
df_clean_log["MILEAGE_KM"] = df_clean_log['MILEAGE_KM'].apply(np.log)

df_clean_log["DEPRE_VALUE_PER_YEAR"] = df_clean_log['DEPRE_VALUE_PER_YEAR'].apply(np.log)
df_clean_log["ROAD_TAX_PER_YEAR"] = df_clean_log['ROAD_TAX_PER_YEAR'].apply(np.log)
df_clean_log["DEREG_VALUE_FROM_SCRAPE_DATE"] = df_clean_log['DEREG_VALUE_FROM_SCRAPE_DATE'].apply(np.log)
df_clean_log["OMV"] = df_clean_log['OMV'].apply(np.log)
df_clean_log["ARF"] = df_clean_log['ARF'].apply(np.log)
df_clean_log["COE_FROM_SCRAPE_DATE"] = df_clean_log['COE_FROM_SCRAPE_DATE'].apply(np.log)
#df_clean_log["DAYS_OF_COE_LEFT"] = df_clean_log['DAYS_OF_COE_LEFT'].apply(np.log)
df_clean_log["ENGINE_CAPACITY_CC"] = df_clean_log['ENGINE_CAPACITY_CC'].apply(np.log)
df_clean_log["CURB_WEIGHT_KG"] = df_clean_log['CURB_WEIGHT_KG'].apply(np.log)



In [15]:
#Drop 'ARF', 'ROAD_TAX_PER_YEAR','REG_DATE','MANUFACTURED_YEAR','VEHICLE_TYPE','POST_DATE','CAR_CATEGORY'
df_clean_log.drop('ARF', axis=1, inplace=True)
df_clean_log.drop('ROAD_TAX_PER_YEAR', axis=1, inplace=True)

#Dropping columns not needed in our regression as they have been encoded 
df_clean_log.drop('BRAND', axis=1, inplace=True)
df_clean_log.drop('REG_DATE', axis=1, inplace=True)
df_clean_log.drop('MANUFACTURED_YEAR', axis=1, inplace=True)
df_clean_log.drop('VEHICLE_TYPE', axis=1, inplace=True)
df_clean_log.drop('POST_DATE', axis=1, inplace=True)
df_clean_log.drop('CAR_CATEGORY', axis=1, inplace=True)


In [16]:
df_clean_log.to_csv('clean_log_data.csv', index=False)

In [17]:
df_clean_log

,PRICE,DEPRE_VALUE_PER_YEAR,MILEAGE_KM,DEREG_VALUE_FROM_SCRAPE_DATE,OMV,COE_FROM_SCRAPE_DATE,DAYS_OF_COE_LEFT,ENGINE_CAPACITY_CC,CURB_WEIGHT_KG,NO_OF_OWNERS,...,VEHICLE_TYPE_Others,VEHICLE_TYPE_SUV,VEHICLE_TYPE_Sports Car,VEHICLE_TYPE_Stationwagon,CAR_CATEGORY_ECONOMY,CAR_CATEGORY_EXOTIC,CAR_CATEGORY_LUXURY,CAR_CATEGORY_MID_LEVEL,CAR_CATEGORY_OTHERS,CAR_CATEGORY_ULTRA_LUXURY
0,80800.0,10.022159,11.703546,10.856920,10.877273,10.913633,762.0,7.596392,7.426549,2.0,...,0,0,0,0,0,0,1,0,0,0
2,130800.0,9.947026,10.275051,10.732214,10.335270,10.488660,1976.0,7.494430,7.265430,2.0,...,0,0,1,0,0,0,0,1,0,0
4,222800.0,10.292146,10.325482,11.015690,10.534121,10.604132,2466.0,7.821242,7.560080,2.0,...,0,0,0,0,1,0,0,0,0,0
5,98800.0,9.626416,12.358794,9.992871,10.588628,10.421329,2375.0,7.592870,7.387090,3.0,...,0,0,0,0,0,0,1,0,0,0
7,88000.0,9.658865,11.695247,10.346120,9.853772,10.424927,1827.0,7.599902,7.323171,2.0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16586,106800.0,9.540363,11.898188,10.706296,10.893512,10.970334,2798.0,7.493317,7.408531,4.0,...,0,0,0,0,0,0,1,0,0,0
16587,135800.0,9.694000,11.374996,11.145031,9.897168,11.156279,2827.0,7.598399,7.370231,1.0,...,0,1,0,0,1,0,0,0,0,0
16588,162000.0,9.815749,12.013701,11.463483,11.347555,11.586659,2920.0,8.188133,7.652071,2.0,...,0,1,0,0,0,0,0,0,0,1
16589,98800.0,9.552511,11.599103,10.307218,10.725490,10.661556,2559.0,7.999343,7.293018,6.0,...,0,0,1,0,0,0,1,0,0,0
